In [1]:
#imports
import math
import numpy as np
import random
import itertools
from random import choices
import cmath


In [2]:
#constants
GAIN_MATRIX = (
    (3.95370716e-13, 1.40831353e-12, 1.30345221e-13, 1.58616128e-12),
    (1.10454943e-11, 1.08797904e-13, 1.70626457e-11, 1.87910752e-13),
    (5.35349571e-12, 1.75915771e-12, 1.10933550e-11, 2.15002508e-12),
    (9.57092193e-12, 1.14512900e-11, 7.62246884e-13, 1.58625393e-11)
)

SUM_GAIN = (
    2.63652827e-11, 1.47275591e-11, 2.90485928e-11, 1.97866364e-11
)

TARGET_BITRATE=13142071.76687619

D=10**6  #bandwidth
N=-174 + 10 * math.log10(D) #noise in dbm

In [3]:
#generating population in dbm
def generate_random_solutions(powers, no_of_users, num_random_solutions):
    return np.random.choice(powers, size=(num_random_solutions, no_of_users))


In [4]:
#usage
powers = [0,2,4]
no_of_users = 4
num_random_solutions = 8
pop_size_dbm = generate_random_solutions(powers, no_of_users, num_random_solutions)
print(pop_size_dbm)

[[4 4 4 0]
 [4 4 4 2]
 [2 4 2 4]
 [4 0 4 4]
 [2 4 0 4]
 [0 4 4 4]
 [4 4 4 0]
 [0 2 2 0]]


In [5]:
# converting population to linear watt
def calculate_linear_trans_power(pop_size_dbm):
    result=(pow(10, -3) * np.power(10.0, np.array(pop_size_dbm, dtype="float") / 10))
    return result


In [6]:
#usage
linear_pop=calculate_linear_trans_power(pop_size_dbm)
print(linear_pop)

[[0.00251189 0.00251189 0.00251189 0.001     ]
 [0.00251189 0.00251189 0.00251189 0.00158489]
 [0.00158489 0.00251189 0.00158489 0.00251189]
 [0.00251189 0.001      0.00251189 0.00251189]
 [0.00158489 0.00251189 0.001      0.00251189]
 [0.001      0.00251189 0.00251189 0.00251189]
 [0.00251189 0.00251189 0.00251189 0.001     ]
 [0.001      0.00158489 0.00158489 0.001     ]]


In [7]:
#usage for converting the noise to linear
linear_noise = calculate_linear_trans_power([N])
print("Linear noise:", linear_noise)

Linear noise: [3.98107171e-15]


In [8]:
def calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise):
    a = np.array(linear_pop)
    b = np.array(SUM_GAIN)
    bit_rate_matrix = (D * np.log2(1 + ((a * b) / linear_noise)))
    return bit_rate_matrix

In [9]:
#Usage for calculating the bitrates through big formula
bitrate=calculate_formula_and_sums(linear_pop, SUM_GAIN, linear_noise)
print(bitrate)

[[4140399.83799027 3363516.1582802  4272652.02625749 2577774.03041514]
 [4140399.83799027 3363516.1582802  4272652.02625749 3150104.99872635]
 [3523086.17671602 3363516.1582802  3651275.96911754 3753232.64345308]
 [4140399.83799027 2232475.21650449 4272652.02625749 3753232.64345308]
 [3523086.17671602 3363516.1582802  3052533.55366803 3753232.64345308]
 [2930294.44975843 3363516.1582802  4272652.02625749 3753232.64345308]
 [4140399.83799027 3363516.1582802  4272652.02625749 2577774.03041514]
 [2930294.44975843 2778870.2310227  3651275.96911754 2577774.03041514]]


In [10]:
def row_wise_sums(matrix):
    return [sum(row) for row in matrix]

In [11]:
# USAGE
row_sum_bitrate = row_wise_sums(bitrate)
# print(row_sum_bitrate)
print(*row_sum_bitrate, sep='\n')
print(type(row_sum_bitrate))

14354342.052943096
14926673.021254312
14291110.947566848
14398759.724205334
13692368.532117337
14319695.277749203
14354342.052943096
11938214.680313796
<class 'list'>


In [12]:
def get_sorted_row_sums(matrix):
    # Calculate the sum of each row and store it as a tuple (row_sum, row)
    row_sums = [(sum(row), row) for row in matrix]

    # Sort the row sums in increasing order
    sorted_row_sums = sorted(row_sums, key=lambda x: x[0])

    # Return the sorted row sums
    return [row_sum for row_sum, _ in sorted_row_sums]

In [13]:
sorted_row_sums = get_sorted_row_sums(bitrate)
print("Row sums in increasing order:")
print(*sorted_row_sums, sep='\n')

Row sums in increasing order:
11938214.680313796
13692368.532117337
14291110.947566848
14319695.277749203
14354342.052943096
14354342.052943096
14398759.724205334
14926673.021254312


In [14]:
def get_sorted_matrix(sorted_row_sums, matrix):
    # Create a dictionary with row sums as keys and corresponding rows as values
    row_sum_to_row = {sum(row): row for row in matrix}

    # Get the corresponding matrix rows based on the sorted row sums
    sorted_matrix = [row_sum_to_row[row_sum] for row_sum in sorted_row_sums]

    return sorted_matrix



sorted_matrix = get_sorted_matrix(sorted_row_sums, bitrate)

print("Corresponding matrix rows:")
for row in sorted_matrix:
    print(row)



Corresponding matrix rows:
[2930294.44975843 2778870.2310227  3651275.96911754 2577774.03041514]
[3523086.17671602 3363516.1582802  3052533.55366803 3753232.64345308]
[3523086.17671602 3363516.1582802  3651275.96911754 3753232.64345308]
[2930294.44975843 3363516.1582802  4272652.02625749 3753232.64345308]
[4140399.83799027 3363516.1582802  4272652.02625749 2577774.03041514]
[4140399.83799027 3363516.1582802  4272652.02625749 2577774.03041514]
[4140399.83799027 2232475.21650449 4272652.02625749 3753232.64345308]
[4140399.83799027 3363516.1582802  4272652.02625749 3150104.99872635]
